In [ ]:
import pandas as pd
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.decomposition import PCA
from sklearn.cluster import KMeans
from sklearn.metrics import adjusted_rand_score, normalized_mutual_info_score
from sklearn.metrics import (
    silhouette_score,
    davies_bouldin_score,
    calinski_harabasz_score,
    adjusted_rand_score,
    normalized_mutual_info_score,
    homogeneity_completeness_v_measure,
    confusion_matrix
)

In [ ]:
def evaluate_clustering(X, clusters, y_true=None, label_names=None):
    """
    X: array-like (n_samples, n_features) - usually scaled/PCA output used for clustering
    clusters: array-like (n_samples,) - predicted cluster id for each sample
    y_true: array-like (n_samples,) or None - optional true labels (e.g., genre_label) for external eval
    label_names: list[str] or None - optional class names aligned with y_true encoding (e.g., le.classes_)
    """
    results = {}

    # ---- Internal / unsupervised metrics ----
    # Note: silhouette requires at least 2 clusters and no all-same labels.
    unique_clusters = np.unique(clusters)
    if len(unique_clusters) > 1 and not np.all(clusters == clusters[0]):
        results["silhouette"] = silhouette_score(X, clusters)
    else:
        results["silhouette"] = np.nan

    # Davies-Bouldin and Calinski-Harabasz also require >1 cluster
    if len(unique_clusters) > 1:
        results["davies_bouldin"] = davies_bouldin_score(X, clusters)
        results["calinski_harabasz"] = calinski_harabasz_score(X, clusters)
    else:
        results["davies_bouldin"] = np.nan
        results["calinski_harabasz"] = np.nan

    # ---- External / label-based metrics (optional) ----
    if y_true is not None:
        results["ARI"] = adjusted_rand_score(y_true, clusters)
        results["NMI"] = normalized_mutual_info_score(y_true, clusters)
        h, c, v = homogeneity_completeness_v_measure(y_true, clusters)
        results["homogeneity"] = h
        results["completeness"] = c
        results["v_measure"] = v

        # Confusion-like matrix: rows=true genre, cols=cluster
        cm = confusion_matrix(y_true, clusters)
        cm_df = pd.DataFrame(
            cm,
            index=label_names if label_names is not None else [f"genre_{i}" for i in np.unique(y_true)],
            columns=[f"cluster_{c}" for c in np.unique(clusters)]
        )

        # Cluster -> majority genre mapping (interpretable)
        tmp = pd.DataFrame({"y_true": y_true, "cluster": clusters})
        cluster_to_majority_label = (
            tmp.groupby("cluster")["y_true"]
               .agg(lambda s: s.value_counts().idxmax())
        )

        if label_names is not None:
            cluster_to_majority_name = cluster_to_majority_label.map(lambda k: label_names[k])
        else:
            cluster_to_majority_name = cluster_to_majority_label

        return results, cm_df, cluster_to_majority_name

    return results, None, None

In [ ]:
df = pd.read_csv(r"C:\Users\zuhai\Desktop\Projects\Machine Learning Project\songs_data.csv")
le = LabelEncoder()
df["genre_label"] = le.fit_transform(df["genre"])

# Optional: see the mapping
genre_mapping = dict(zip(le.classes_, le.transform(le.classes_)))
print("Genre mapping:", genre_mapping)

In [ ]:
audio_features = [
    "danceability", "energy", "key", "loudness", "mode", "speechiness",
    "acousticness", "instrumentalness", "liveness", "valence",
    "tempo", "duration_ms"
]

# Keep a clean copy with needed cols (genre kept ONLY for post-hoc inspection)
cols_to_keep = audio_features + (["genre"] if "genre" in df.columns else [])
df_clean = df[cols_to_keep].dropna().reset_index(drop=True)

X = df_clean[audio_features].values  # <-- no genre here


# ----------------------------
# 2) Scale + PCA (recommended)
# ----------------------------
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# Keep 90% variance; you can change to n_components=2 for easy plotting
pca = PCA(n_components=0.90, random_state=42)
X_pca = pca.fit_transform(X_scaled)


# ---------------------------------------------------------
# 3) Choose K WITHOUT using genre labels (silhouette search)
# ---------------------------------------------------------
def choose_best_k(X_embedded, k_min=2, k_max=10, random_state=42):
    best_k, best_score = None, -1
    scores = {}

    for k in range(k_min, k_max + 1):
        km = KMeans(n_clusters=k, random_state=random_state, n_init="auto")
        labels = km.fit_predict(X_embedded)

        # silhouette needs at least 2 clusters and no singleton-only issues
        score = silhouette_score(X_embedded, labels)
        scores[k] = score

        if score > best_score:
            best_score = score
            best_k = k

    return best_k, scores


best_k, sil_scores = choose_best_k(X_pca, k_min=2, k_max=10)
print("Silhouette scores by K:", sil_scores)
print("Best K selected (by silhouette):", best_k)


# ----------------------------
# 4) K-Means clustering
# ----------------------------
kmeans = KMeans(n_clusters=best_k, random_state=42, n_init="auto")
kmeans_labels = kmeans.fit_predict(X_pca)
df_clean["kmeans_cluster"] = kmeans_labels


# ----------------------------
# 5) Hierarchical clustering
# ----------------------------
# Ward linkage works well with Euclidean distance on scaled/PCA data
hier = AgglomerativeClustering(n_clusters=best_k, linkage="ward")
hier_labels = hier.fit_predict(X_pca)
df_clean["hier_cluster"] = hier_labels


# ---------------------------------------------
# 6) Internal (label-free) cluster evaluation
# ---------------------------------------------
def internal_metrics(X_embedded, labels, name="model"):
    uniq = np.unique(labels)
    if len(uniq) < 2:
        return {
            "model": name,
            "silhouette": np.nan,
            "davies_bouldin": np.nan,
            "calinski_harabasz": np.nan,
            "n_clusters": len(uniq)
        }

    return {
        "model": name,
        "silhouette": silhouette_score(X_embedded, labels),
        "davies_bouldin": davies_bouldin_score(X_embedded, labels),
        "calinski_harabasz": calinski_harabasz_score(X_embedded, labels),
        "n_clusters": len(uniq)
    }


kmeans_metrics = internal_metrics(X_pca, kmeans_labels, "KMeans")
hier_metrics = internal_metrics(X_pca, hier_labels, "Hierarchical")

metrics_df = pd.DataFrame([kmeans_metrics, hier_metrics])
print("\nInternal evaluation (no genre used):")
print(metrics_df)


# --------------------------------------------------------------------
# 7) Post-hoc investigation: Are clusters "meaningful" w.r.t. genre?
#     (This does NOT affect training; it is just analysis.)
# --------------------------------------------------------------------
if "genre" in df_clean.columns:
    # (a) Genre distribution per cluster (normalized)
    kmeans_genre_dist = (
        df_clean.groupby("kmeans_cluster")["genre"]
                .value_counts(normalize=True)
                .unstack(fill_value=0)
                .round(3)
    )

    hier_genre_dist = (
        df_clean.groupby("hier_cluster")["genre"]
                .value_counts(normalize=True)
                .unstack(fill_value=0)
                .round(3)
    )

    print("\nK-Means: cluster -> genre distribution (normalized):")
    print(kmeans_genre_dist)

    print("\nHierarchical: cluster -> genre distribution (normalized):")
    print(hier_genre_dist)

    # (b) Majority-genre label per cluster (easy interpretation)
    kmeans_majority = (
        df_clean.groupby("kmeans_cluster")["genre"]
                .agg(lambda s: s.value_counts().idxmax())
    )

    hier_majority = (
        df_clean.groupby("hier_cluster")["genre"]
                .agg(lambda s: s.value_counts().idxmax())
    )

    print("\nK-Means: cluster -> majority genre")
    print(kmeans_majority)

    print("\nHierarchical: cluster -> majority genre")
    print(hier_majority)


# -------------------------------------------------
# 8) Interpret clusters by mean audio feature values
# -------------------------------------------------
kmeans_feature_means = df_clean.groupby("kmeans_cluster")[audio_features].mean().round(3)
hier_feature_means = df_clean.groupby("hier_cluster")[audio_features].mean().round(3)

print("\nK-Means: mean audio features per cluster")
print(kmeans_feature_means)

print("\nHierarchical: mean audio features per cluster")
print(hier_feature_means)


# ----------------------------
# 9) Optional: quick PCA plot
# ----------------------------
# If you want a visual scatter plot:
# import matplotlib.pyplot as plt
# plt.figure()
# plt.scatter(X_pca[:, 0], X_pca[:, 1], c=kmeans_labels, s=10)
# plt.title("K-Means Clusters (PCA space)")
# plt.xlabel("PC1")
# plt.ylabel("PC2")
# plt.show()